In [1]:
import sys
sys.path.append('/vol/tensusers4/nhollain/thesis2023-2024/s_clip_scripts')
sys.path.append('./scripts')
sys.path.append('./probvlm')

import itertools
from collections import Counter
from main import main, format_checkpoint
from params import parse_args
import copy
import os
from tqdm import tqdm
from datetime import datetime

from tuning_tools import prep_str_args, evaluate_checkpoint

[nltk_data] Downloading package punkt to C:\Users\Laptop of
[nltk_data]     Natalie\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Laptop of
[nltk_data]     Natalie\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
########################
# Count the models that have been trained and evaluated already, based on their parameters 
results = []
if os.path.exists('./results/test_eval_ilt.txt'):   
    with open('./results/test_eval_ilt.txt', 'r') as f:
        results = f.readlines()

# Remove any non-result lines from the eval file, and split the lines on the tab character
# (results have format: model_name\tdataset_name\tmetric_name\tmetric_value)
results = [r.replace('\n','').split('\t')[0] for r in results if '\t' in r]
model_names = results
# Remove the timestamp from the model names, as well as the specific fold - rest of the name contains params
model_names = ['-'.join(m.split('-')[2:]).split('-fold')[0] for m in model_names]
model_names = dict(Counter(model_names))
# print('Model_names', model_names)

# Do a grid search on the parameters
# NOTE: for active learning, save-freq should be set to 1
base_str_args = ''' --train-data ILT
--test-data ILT
--keyword-path keywords/RS/class-name.txt
--zeroshot-frequency 5  
--method base
--lr 5e-5
--eval-file ./results/eval_ilt.txt
'''
# --active-learning
# --probvlm
#--save-freq 1

# Dictionary of values to gridsearch for hyperparam tuning
gridsearch_dict = {
    '--epochs' : [5], #list(range(15,36,5)) if 'active-learning' in base_str_args else [35], #[10,15,20,25,30,35],
    '--batch-size' : [64],
    '--al-iter': [5], #list(range(3,17,2)), #list(range(1,6,2)),
    '--al-epochs': [10],
    '--label-ratio': [0.05, 0.1, 0.2, 0.4, 0.8],
    #'--pl-method': ['soft.text'],
}

# How many times to re-evaluate the model on the test set (to get an average and std of the results)
num_repeats = 5
num_evals = 20 # How many evaluations are done with evaluate_checkpoint(...) - KEEP THIS FIXED

gridsearch_values = list(gridsearch_dict.values())
gridsearch_keys = list(gridsearch_dict.keys())
configs = list(itertools.product(*gridsearch_values))
print('Number of configs:', len(configs))

Number of configs: 5


In [3]:
t_start = datetime.now() 
for c, config in enumerate(configs): # Gridsearch
    str_args = copy.deepcopy(base_str_args)
    # Add the gridsearch parameters to the arguments
    for i, param in enumerate(config):
        param_name = gridsearch_keys[i]
        str_args += '\n{} {}'.format(param_name, param)
        
    str_args = prep_str_args(str_args)
    print(str_args)
    args = parse_args(str_args)
    checkpoint_hypothetical = format_checkpoint(args)
    # Remove the timestamp from the hypothetical checkpoint, so we can compare to the params of other checkpoints
    checkpoint_params = '-'.join(checkpoint_hypothetical.split('-')[2:]).split('-fold')[0]

    # Check if we've already trained the exact same model, correct the number of training iterations we still need to do
    if checkpoint_params in model_names:
        # The number of times to repeat depends on how often the model's been evaluated already
        start_repeat = int(model_names[checkpoint_params]/num_evals)
    else: # If we've never trained + evaluated the model before, just use num_repeats
        start_repeat = 0
    print(f'Config number {c}: {max(0,num_repeats-start_repeat)} repeats')
    for i in range(start_repeat, num_repeats):
        args = parse_args(str_args)
        args.seed = i
        # We compute here for which epochs we need to evaluate (based on for which epochs we checkpoint)
        epoch_freq = args.save_freq
        epochs = list(range(epoch_freq,args.epochs+1,epoch_freq))
        
        checkpoint_path = main(args) # Calls the main.py function of S-CLIP
        for epoch in epochs:
            evaluate_checkpoint(checkpoint_path, epoch = epoch, split = 'test', eval_file = './results/eval_ilt.txt')
        # Remove the checkpoint after evaluating, to save space
        os.system(f"rm -r {checkpoint_path}")  

t_delta = datetime.now() - t_start   
print(f'Elapsed time: {t_delta}')

['--train-data', 'ILT', '--test-data', 'ILT', '--keyword-path', 'keywords/RS/class-name.txt', '--zeroshot-frequency', '5', '--method', 'base', '--lr', '5e-5', '--eval-file', './results/eval_ilt.txt', '--epochs', '5', '--batch-size', '64', '--al-iter', '5', '--al-epochs', '10', '--label-ratio', '0.05']
Config number 0: 5 repeats


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:03<00:00,  1.62it/s]


KeyboardInterrupt: 